In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import pandas as pd 
import cv2
import json
import tensorflow as tf
import matplotlib.pyplot as plt
import albumentations as albu
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix
import tensorflow.keras.backend as K
from tensorflow.keras import Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.densenet import DenseNet169, DenseNet121
import efficientnet.tfkeras as efn
from clr import OneCycleLR
print('tensorflow version:', tf.__version__)
try:
    print('available GPU devices:', len(os.environ['CUDA_VISIBLE_DEVICES']), 
          '| device num:', os.environ['CUDA_VISIBLE_DEVICES'])
except:
    pass
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [ ]:
from tensorflow.keras.utils import get_custom_objects
class Mish(Activation):
    '''
    Mish Activation Function.
    .. math::
        mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + e^{x}))
    Shape:
        - Input: Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
        - Output: Same shape as the input.
    Examples:
        >>> X = Activation('Mish', name="conv1_act")(X_input)
    '''
    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'
def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))
get_custom_objects().update({'Mish': Mish(mish)})

In [ ]:
MAIN_PATH = '.'
DATA_PATH = '{}/data'.format(MAIN_PATH)
MDL_VERSION = 'v0'
MODELS_PATH = '{}/keras_models_{}'.format(MAIN_PATH, MDL_VERSION)
if not os.path.exists(MODELS_PATH):
    os.makedirs(MODELS_PATH)
TILES_PATH = '{}/train_images_tiles_q2_128_16'.format(DATA_PATH)
IMG_SIZE = 128 #224=B0 240=B1 260=B2 300=B3 380=B4 456=B5
SEQ_LEN = 16
BATCH_SIZE = 8
TIFF = 1
RESIZE = None
SEED = 80
print('96x96x16 size:', 96 * 96 * 16)
print('96x96x25 size:', 96 * 96 * 25)
print('128x128x12 size:', 128 * 128 * 12)
print('154x154x10 size:', 154 * 154 * 10)
print('current size:', IMG_SIZE * IMG_SIZE * SEQ_LEN)

In [ ]:
train = pd.read_csv('{}/train.csv'.format(DATA_PATH)).set_index('image_id')
print('train: ', train.shape, '| unique ids:', sum(train['isup_grade'].value_counts()))
files = sorted(set([x[:32] for x in os.listdir(TILES_PATH) if '.ipynb' not in x]))
train = train.loc[files]
train = train.reset_index()
print('train: ', train.shape, '| unique ids:', sum(train['isup_grade'].value_counts()))
train.to_csv('{}/train_dsph.csv'.format(DATA_PATH))

In [ ]:
class DataGenPanda(Sequence):
    def __init__(self, imgs_path, df, batch_size=32, 
                 mode='fit', shuffle=False, aug=None,
                 tiff=-1, resize=None,
                 seq_len=12, img_size=128, n_classes=6):
        self.imgs_path = imgs_path
        self.df = df
        self.shuffle = shuffle
        self.mode = mode
        self.aug = aug
        self.tiff = tiff
        self.resize = resize
        self.batch_size = batch_size
        self.img_size = img_size
        self.seq_len = seq_len
        self.n_classes = n_classes
        self.side = int(seq_len ** .5)
        self.on_epoch_end()
    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    def __getitem__(self, index):
        batch_size = min(self.batch_size, len(self.df) - index * self.batch_size)
        X = np.zeros((batch_size, self.side * self.img_size, self.side * self.img_size, 3), dtype=np.float32)
        imgs_batch = self.df[index * self.batch_size : (index + 1) * self.batch_size]['image_id'].values
        for i, img_name in enumerate(imgs_batch):
            img_patches = self.get_patches(img_name)
            X[i, ] = self.glue_to_one(img_patches)
        if self.mode == 'fit':
            y = np.zeros((self.batch_size, self.n_classes), dtype=np.float32)
            #y = np.zeros(self.batch_size, dtype=np.float32)
            lbls_batch = self.df[index * self.batch_size : (index + 1) * self.batch_size]['isup_grade'].values
            for i in range(self.batch_size):
                y[i, lbls_batch[i]] = 1
                #y[i, ] = lbls_batch[i]
            return X, y
        elif self.mode == 'predict':
            return X
        else:
            raise AttributeError('mode parameter error')
    def get_patches(self, img_name):
        seq_imgs = []
        for i in range(self.seq_len):
            img_path = '{}/{}_{}.png'.format(self.imgs_path, img_name, i)
            img = cv2.imread(img_path)
            if not np.any(img):
                print(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            if self.resize:
                img = cv2.resize(img, (int(img.shape[1] / self.resize), int(img.shape[0] / self.resize)))
            img = img.astype(np.float32) / 255
            if self.aug:
                img = self.aug(image=img)['image']
            seq_imgs.append(img)
        return np.array(seq_imgs).astype(np.float32)
    def glue_to_one(self, imgs_seq):
        img_glue = np.zeros((self.img_size * self.side, self.img_size * self.side, 3), dtype=np.float32)
        for i, ptch in enumerate(imgs_seq):
            x = i // self.side
            y = i % self.side
            img_glue[x * self.img_size : (x + 1) * self.img_size, 
                     y * self.img_size : (y + 1) * self.img_size, :] = ptch
        return img_glue

In [ ]:
X_train, X_val = train_test_split(train, test_size=.2, stratify=train['isup_grade'], random_state=SEED)
lbl_value_counts = X_train['isup_grade'].value_counts()
class_weights = {i: max(lbl_value_counts) / v for i, v in lbl_value_counts.items()}
class_weights

In [ ]:
aug = albu.Compose(
    [
        albu.OneOf(
            [
                albu.RandomBrightness(limit=.15), 
                albu.RandomContrast(limit=.3), 
                albu.RandomGamma()
            ], 
            p=.3
        ),
        albu.HorizontalFlip(p=.3),
        albu.VerticalFlip(p=.3),
        albu.ShiftScaleRotate(shift_limit=.2, scale_limit=.2, rotate_limit=30, p=.3)
    ]
)
train_datagen = DataGenPanda(
    imgs_path=TILES_PATH, 
    df=X_train, 
    batch_size=BATCH_SIZE,
    mode='fit', 
    shuffle=True, 
    aug=aug, 
    tiff=TIFF,
    resize=RESIZE,
    seq_len=SEQ_LEN, 
    img_size=IMG_SIZE, 
    n_classes=6
)
val_datagen = DataGenPanda(
    imgs_path=TILES_PATH, 
    df=X_val, 
    batch_size=BATCH_SIZE,
    mode='fit', 
    shuffle=False, 
    aug=None,
    tiff=TIFF,
    resize=RESIZE,
    seq_len=SEQ_LEN, 
    img_size=IMG_SIZE, 
    n_classes=6
)

In [ ]:
Xt, yt = train_datagen.__getitem__(0)
print('test X: ', Xt.shape)
print('test y: ', yt.shape)
fig, axes = plt.subplots(figsize=(10, 6), ncols=BATCH_SIZE)
for j in range(BATCH_SIZE):
    axes[j].imshow(Xt[j])
    axes[j].axis('off')
    axes[j].set_title(np.argmax(yt[j, ]))
plt.show()
yt

In [ ]:
def qw_kappa_score(y_true, y_pred):
    y_true=tf.math.argmax(y_true, axis=1)
    y_pred=tf.math.argmax(y_pred, axis=1)
    def sklearn_qwk(y_true, y_pred) -> np.float64:
        return cohen_kappa_score(y_true, y_pred, weights='quadratic')
    return tf.compat.v1.py_func(sklearn_qwk, (y_true, y_pred), tf.double)

In [ ]:
bottleneck = efn.EfficientNetB0(
    input_shape=(int(SEQ_LEN ** .5) * IMG_SIZE, int(SEQ_LEN ** .5) * IMG_SIZE, 3),
    weights='imagenet', 
    include_top=False, 
    pooling='avg'
)
bottleneck = Model(inputs=bottleneck.inputs, outputs=bottleneck.layers[-2].output)
model = Sequential()
model.add(bottleneck)
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dropout(.4))
model.add(Dense(512, activation='Mish'))
model.add(BatchNormalization())
model.add(Dropout(.4))
model.add(Dense(128, activation='Mish'))
model.add(BatchNormalization())
model.add(Dropout(.4))
model.add(Dense(6, activation='softmax'))
model.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(lr=1e-2), # epsilon=1e-4 for float16 numerical stability
    metrics=['categorical_accuracy', qw_kappa_score]
)
model.summary()

In [ ]:
%%time
model_file = '{}/model_{}.h5'.format(MODELS_PATH, MDL_VERSION)
if False:
    model = load_model(model_file)
    print('model loaded')
else:
    print('train from scratch')
EPOCHS = 40
earlystopper = EarlyStopping(
    monitor='val_qw_kappa_score', 
    patience=20, 
    verbose=1,
    mode='max'
)
modelsaver = ModelCheckpoint(
    model_file, 
    monitor='val_qw_kappa_score', 
    verbose=1, 
    save_best_only=True,
    mode='max'
)
lrreducer = ReduceLROnPlateau(
    monitor='val_qw_kappa_score',
    factor=.1,
    patience=10,
    verbose=1,
    min_lr=3e-7
)
lrmanager = OneCycleLR(
    max_lr=1e-2,
    end_percentage=0.1, 
    scale_percentage=None,
    maximum_momentum=0.95, 
    minimum_momentum=0.85
)
history = model.fit_generator(
    train_datagen,
    validation_data=val_datagen,
    class_weight=class_weights,
    callbacks=[earlystopper, modelsaver, lrmanager],
    epochs=EPOCHS,
    verbose=1
)

In [ ]:
history_file = '{}/history_{}.txt'.format(MODELS_PATH, MDL_VERSION)
dict_to_save = {}
for k, v in history.history.items():
    dict_to_save.update({k: [np.format_float_positional(x) for x in history.history[k]]})
with open(history_file, 'w') as file:
    json.dump(dict_to_save, file)
ep_max = EPOCHS
plt.plot(history.history['loss'][:ep_max], label='loss')
plt.plot(history.history['val_loss'][:ep_max], label='val_loss')
plt.legend()
plt.show()
plt.plot(history.history['categorical_accuracy'][:ep_max], label='cat acc')
plt.plot(history.history['val_categorical_accuracy'][:ep_max], label='val cat acc')
plt.legend()
plt.show()
plt.plot(history.history['qw_kappa_score'][:ep_max], label='qwk')
plt.plot(history.history['val_qw_kappa_score'][:ep_max], label='val qwk')
plt.legend()
plt.show()

In [ ]:
%%time
model_file = '{}/model_{}.h5'.format(MODELS_PATH, MDL_VERSION)
model = load_model(model_file)
print('model loaded:', model_file)

In [ ]:
%%time
X_score = X_val.copy()
val_datagen = DataGenPanda(
    imgs_path=TILES_PATH, 
    df=X_score, 
    batch_size=1,
    mode='predict', 
    shuffle=False, 
    aug=None,
    tiff=TIFF,
    resize=RESIZE,
    seq_len=SEQ_LEN, 
    img_size=IMG_SIZE, 
    n_classes=6
)
preds = model.predict_generator(val_datagen)

In [ ]:
y_true = X_score['isup_grade'].values
y_pred = np.argmax(preds, axis=1)
print('quadratic cappa score:', cohen_kappa_score(y_true, y_pred, weights='quadratic'))
print('confusion matrix:\n', confusion_matrix(y_true, y_pred))